In [0]:
%python
import pandas as pd
from datetime import datetime, timedelta

# Define the file path
file_path = '/Volumes/workspace/default/spacedata/ELSET_data_08092025.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
display(df.head())

# Convert to Spark DataFrame
df_spark = spark.createDataFrame(df)

# Adjust the table name to include yesterday's date
yesterday = datetime.utcnow() - timedelta(days=1)
table_name = f"gp_latest_{yesterday.strftime('%Y%m%d_%H%M%S')}s"

# Save as Delta table
df_spark.write.format("delta").mode("overwrite").saveAsTable(table_name)

In [0]:
%python
import requests
import pandas as pd
from io import StringIO
from datetime import datetime

USERNAME = "dristy.changkakoti@gmail.com"
PASSWORD = "SouthJordan1!*5"

# Space-Track query: latest 30 days of GP data in CSV format
URL = (
    "https://www.space-track.org/basicspacedata/query/class/gp"
    "/EPOCH/%3Enow-30"
    "/orderby/NORAD_CAT_ID,EPOCH"
    "/format/csv"
)

def fetch_latest_tle():
    session = requests.Session()

    # Login
    login_url = "https://www.space-track.org/ajaxauth/login"
    resp = session.post(login_url, data={"identity": USERNAME, "password": PASSWORD})
    resp.raise_for_status()

    # Download CSV
    resp = session.get(URL)
    resp.raise_for_status()

    # Convert to DataFrame
    df = pd.read_csv(StringIO(resp.text))
    print(f"Downloaded {len(df)} records")

    # Save as Delta table
    # delta_path = "/Volumes/workspace/default/spacedata/gp_latest_delta"
    df_spark = spark.createDataFrame(df)
    table_name = f"gp_latest_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}s"
    df_spark.write.format("delta").mode("overwrite").saveAsTable(table_name)

    return df

if __name__ == "__main__":
    fetch_latest_tle()